# `scinum` example

In [1]:
! [ -f "scinum.py" ] || pip install scinum
from scinum import Number, Correlation, NOMINAL, UP, DOWN

The examples below demonstrate

- [Numbers and formatting](#Numbers-and-formatting)
- [Defining uncertainties](#Defining-uncertainties)
- [Multiple uncertainties](#Multiple-uncertainties)
- [Configuration of correlations](#Configuration-of-correlations)
- [Automatic uncertainty propagation](#Automatic-uncertainty-propagation)
- [Rounding](#Rounding)

### Numbers and formatting

In [2]:
n = Number(1.234, 0.2)
n

<Number at 0x106724130, '1.234 +- 0.2'>

The uncertainty definition is absolute. See the examples with [multiple uncertainties](#Multiple-uncertainties) for relative uncertainty definitions.

The representation of numbers (`repr`) in jupyter notebooks uses latex-style formatting. Internally, [`Number.str()`](https://scinum.readthedocs.io/en/latest/#scinum.Number.str) is called, which - among others - accepts a `format` argument, defaulting to `"%s"` (configurable globally or per instance via [`Number.default_format`](https://scinum.readthedocs.io/en/latest/#scinum.Number.default_format)). Let's change the format for this instance:

In [3]:
n.default_format = "%.2f"
n

<Number at 0x106724130, '1.23 +- 0.20'>

In [4]:
# or
n.str("%.3f")

'1.234 +- 0.200'

### Defining uncertainties

Above, `n` is defined with a single, symmetric uncertainty. Here are some basic examples to access and play it:

In [5]:
# nominal value
print(n.nominal)
print(type(n.nominal))

1.234
<class 'float'>


In [6]:
# get the uncertainty
print(n.get_uncertainty())
print(n.get_uncertainty(direction=UP))
print(n.get_uncertainty(direction=DOWN))

(0.2, 0.2)
0.2
0.2


In [7]:
# get the nominal value, shifted by the uncertainty
print(n.get())      # nominal value
print(n.get(UP))    # up variation
print(n.get(DOWN))  # down variation

1.234
1.434
1.034


In [8]:
# some more advanved use-cases:

# 1. get the multiplicative factor that would scale the nomninal value to the UP/DOWN varied ones
print("absolute factors:")
print(n.get(UP, factor=True))
print(n.get(DOWN, factor=True))

# 2. get the factor to obtain the uncertainty only (i.e., the relative unceratinty)
# (this is, of course, more useful in case of multiple uncertainties, see below)
print("\nrelative factors:")
print(n.get(UP, factor=True, unc=True))
print(n.get(DOWN, factor=True, unc=True))

absolute factors:
1.1620745542949757
0.8379254457050244

relative factors:
0.1620745542949757
0.1620745542949757


There are also a few shorthands for the above methods:

In [9]:
# __call__ is forwarded to get()
print(n())
print(n(UP))

# u() is forwarded to get_uncertainty()
print(n.u())
print(n.u(direction=UP))

1.234
1.434
(0.2, 0.2)
0.2


### Multiple uncertainties


Let's create a number that has two uncertainties: `"stat"` and `"syst"`. The `"stat"` uncertainty is asymmetric, and the `"syst"` uncertainty is relative. Float and integer values are interpreted as absolute uncertainties, imaginary parts of complex numbers as relative ones (= just append `j`, e.g. `0.5j`).

In [10]:
n = Number(8848, {
    "stat": (30, 20),  # absolute +30-20 uncertainty
    "syst": 0.5j,      # relative +-50% uncertainty
})
n

<Number at 0x10673af40, '8848.0 +30.0-20.0 (stat) +- 4424.0 (syst)'>

Similar to above, we can access the uncertainties and shifted values with [`get()`](https://scinum.readthedocs.io/en/latest/#scinum.Number.get) (or `__call__`) and [`get_uncertainty()`](https://scinum.readthedocs.io/en/latest/#scinum.Number.get_uncertainty) (or [`u()`](https://scinum.readthedocs.io/en/latest/#scinum.Number.u)). But this time, we can distinguish between the combined (in quadrature) value or the particular uncertainty sources:

In [11]:
# nominal value as before
print(n.nominal)

# get all uncertainties (stored absolute internally)
print(n.uncertainties)

8848.0
OrderedDict([('stat', (30.0, 20.0)), ('syst', (4424.0, 4424.0))])


In [12]:
# get particular uncertainties
print(n.u("syst"))
print(n.u("stat"))
print(n.u("stat", direction=UP))

(4424.0, 4424.0)
(30.0, 20.0)
30.0


In [13]:
# get the nominal value, shifted by particular uncertainties
print(n(UP, "stat"))
print(n(DOWN, "syst"))

# compute the shifted value for both uncertainties, added in quadrature without correlation (default but configurable)
print(n(UP))

8878.0
4424.0
13272.101716733014


As before, we can also access certain aspects of the uncertainties:

In [14]:
print("factors for particular uncertainties:")
print(n.get(UP, "stat", factor=True))
print(n.get(DOWN, "syst", factor=True))

print("\nfactors for the combined uncertainty:")
print(n.get(UP, factor=True))
print(n.get(DOWN, factor=True))

factors for particular uncertainties:
1.0033905967450272
0.5

factors for the combined uncertainty:
1.500011496014129
0.49999489062775576


We can also apply some nice formatting:

In [15]:
print(n.str())
print(n.str("%.2f"))
print(n.str("%.2f", unit="m"))
print(n.str("%.2f", unit="m", force_asymmetric=True))
print(n.str("%.2f", unit="m", scientific=True))
print(n.str("%.2f", unit="m", si=True))
print(n.str("%.2f", unit="m", style="root"))
print(n.str("%.2f", unit="m", style="latex"))

8848.0 +30.0-20.0 (stat) +- 4424.0 (syst)
8848.00 +30.00-20.00 (stat) +- 4424.00 (syst)
8848.00 +30.00-20.00 (stat) +- 4424.00 (syst) m
8848.00 +30.00-20.00 (stat) +4424.00-4424.00 (syst) m
8.85 +0.03-0.02 (stat) +- 4.42 (syst) x 1E3 m
8.85 +0.03-0.02 (stat) +- 4.42 (syst) km
8848.00 ^{+30.00}_{-20.00} #left(stat#right) #pm 4424.00 #left(syst#right) m
8848.00 \,^{+30.00}_{-20.00} \left(\text{stat}\right) \pm 4424.00 \left(\text{syst}\right)\,m


### Configuration of correlations

Let's assume that we have a second measurement for the quantity `n` we defined above,

In [16]:
n

<Number at 0x10673af40, '8848.0 +30.0-20.0 (stat) +- 4424.0 (syst)'>

and we measured it with the same sources of uncertainty,

In [17]:
n2 = Number(8920, {
    "stat": (35, 15),  # absolute +35-15 uncertainty
    "syst": 0.3j,      # relative +-30% uncertainty
})
n2

<Number at 0x106747b50, '8920.0 +35.0-15.0 (stat) +- 2676.0 (syst)'>

 Now, we want to compute the average measurement, including correct error propagation under consideration of sensible correlations. For more info on automatic uncertainty propagation, see the [subsequent section](#Automatic-uncertainty-propagation).
 
In this example, we want to fully correlate the *systematic* uncertainty, whereas we can treat *statistical* effects as uncorrelated. However, just wirting `(n + n2) / 2` will consider equally named uncertainty sources to be 100% correlated, i.e., both `syst` and `stat` uncertainties will be simply averaged. This is the default behavior in scinum as it is not possible (nor wise) to *guesstimate* the meaning of an uncertainty from its name.

While this approach is certainly correct for `syst`, we don't achieve the correct treatment for `stat`:

In [18]:
(n + n2) / 2

<Number at 0x10674c5b0, '8884.0 +- 3550.0 (syst) +32.5-17.5 (stat)'>

Instead, we need to define the correlation specifically for `stat`. This can be achieved in multiple ways, but the most pythonic way is to use a [`Correlation`](https://scinum.readthedocs.io/en/latest/#correlation) object.

In [19]:
(n @ Correlation(stat=0) + n2) / 2

<Number at 0x10674cfa0, '8884.0 +- 3550.0 (syst) +23.04886114323222-12.500000000000002 (stat)'>

**Note** that the statistical uncertainty decreased as desired, whereas the systematic one remained the same.
`Correlation` objects have a default value that can be set as the first positional, yet optional parameter, and itself defaults to one.

Internally, the operation `n @ Correlation(stat=0)` (or `n * Correlation(stat=0)` in Python 2) is evaluated prior to the addition of `n2` and generates a so-called [`DeferredResult`](https://scinum.readthedocs.io/en/latest/#deferredresult). This object carries the information of `n` and the correlation over to the next operation, at which point the uncertainty propagation is eventually resolved. As usual, in situations where the operator precedence might seem unclear, it is recommended to use parentheses to structure the expression.

### Automatic uncertainty propagation

Let's continue working with the number `n` from above.

Uncertainty propagation works in a pythonic way:

In [20]:
n + 200

<Number at 0x10674c9d0, '9048.0 +- 4424.0 (syst) +30.0-20.0 (stat)'>

In [21]:
n / 2

<Number at 0x10674ceb0, '4424.0 +- 2212.0 (syst) +15.0-10.0 (stat)'>

In [22]:
n**0.5

<Number at 0x1068440a0, '94.06380813043877 +- 23.515952032609693 (syst) +0.15946622083596085-0.10631081389064057 (stat)'>

In cases such as the last one, formatting makes a lot of sense ...

In [23]:
(n**0.5).str("%.2f")

'94.06 +- 23.52 (syst) +0.16-0.11 (stat)'

More complex operations such as `exp`, `log`, `sin`, etc, are provided on the `ops` object, which mimics Python's `math` module. The benefit of the `ops` object is that all its operations are aware of Gaussian error propagation rules.

In [24]:
from scinum import ops

# change the default format for convenience
Number.default_format = "%.3f"

# compute the log of n
ops.log(n)

<Number at 0x1068448e0, '9.088 +0.003-0.002 (stat) +- 0.500 (syst)'>

The propagation is actually performed simultaneously per uncertainty source.

In [25]:
m = Number(5000, {"syst": 1000})

n + m

<Number at 0x1068447f0, '13848.000 +- 5424.000 (syst) +30.000-20.000 (stat)'>

In [26]:
n / m

<Number at 0x106844970, '1.770 +- 0.531 (syst) +0.006-0.004 (stat)'>

As described [above](#Configuration-of-correlations), equally named uncertainty sources are assumed to be fully correlated. You can configure the correlation in operations through `Correlation` objects, or by using explicit methods on the number object.

In [27]:
# n.add(m, rho=0.5, inplace=False)

# same as
n @ Correlation(0.5) + m

<Number at 0x106849040, '13848.000 +- 4999.578 (syst) +30.000-20.000 (stat)'>

When you set `inplace` to `True` (the default), `n` is updated inplace.

In [28]:
n.add(m, rho=0.5)
n

<Number at 0x10673af40, '13848.000 +- 4999.578 (syst) +30.000-20.000 (stat)'>

### Rounding

The above examples used format strings such as `"%.2f"` or `"%.3f"` to enforce a certain amount of significant digits after the decimal point when the number is printed. Internally, this is done via [`round_value`](https://scinum.readthedocs.io/en/latest/#round-value) (and [`round_uncertainty`](https://scinum.readthedocs.io/en/latest/#round-uncertainty)) which provide a variety of automatic rounding options that are accepted by both [`Number.default_format`](https://scinum.readthedocs.io/en/latest/#scinum.Number.default_format) and [`Number.str()`](https://scinum.readthedocs.io/en/latest/#scinum.Number.str):

- Format strings (e.g. `"%.2f"`) allow to round values and their uncertainties in a pythonic way.
- Negative integers or 0 are interpreted similarly to format strings and control the number of significant digits *after* the decimal point. For instance, `-2` is identical to `"%.2f"`.
- Positive integers define the total number of significant digits, evaluated on the smallest value across nominal and uncertainty values.

In addition, there are three methods that implement uncertainty-based rounding rules:
- `"pdg"`: Applies rounding rules defined by the [Particle Data Group](https://pdg.lbl.gov/2021/reviews/rpp2021-rev-rpp-intro.pdf#page=18) based on the values of the three leading significant digits.
- `"pdg+1"`: Same as `"pdg"` with one additional digit.
- `"publication"`: Same as `"pdg+1"` but it does not apply the rounding of values above 949 to 1000 (see the link above).

All rounding rules are demonstrated below.

In [29]:
n = Number(1.234, {"a": 0.22, "b": 0.533, "c": 0.97})
n
print(n.str())

1.234 +- 0.220 (a) +- 0.533 (b) +- 0.970 (c)


In [30]:
# format string
n.str("%.2f")

'1.23 +- 0.22 (a) +- 0.53 (b) +- 0.97 (c)'

In [31]:
# negative integer -> same as format string
n.str(-2)

'1.23 +- 0.22 (a) +- 0.53 (b) +- 0.97 (c)'

In [32]:
# positive integer -> uses unc. "c" with the smallest value as reference and picks 1 digit
n.str(1)

'1.2 +- 0.2 (a) +- 0.5 (b) +- 1.0 (c)'

In [33]:
# zero -> integer rounding
n.str(0)

'1 +- 0 (a) +- 1 (b) +- 1 (c)'

In [34]:
# "pdg" -> based on three leading digits of uncertainty
print(Number(1.234, 0.22).str("pdg"))
print(Number(1.234, 0.53).str("pdg"))
print(Number(1.234, 0.97).str("pdg"))

1.23 +- 0.22
1.2 +- 0.5
1.2 +- 1.0


In [35]:
# "pdg+1" -> "pdg" with one additional digit
print(Number(1.234, 0.22).str("pdg+1"))
print(Number(1.234, 0.53).str("pdg+1"))
print(Number(1.234, 0.97).str("pdg+1"))

1.234 +- 0.220
1.23 +- 0.53
1.23 +- 1.00


In [36]:
# "publication" -> "pdg+1" without rounding above 949 (third line)
print(Number(1.234, 0.22).str("publication"))
print(Number(1.234, 0.53).str("publication"))
print(Number(1.234, 0.97).str("publication"))

1.234 +- 0.220
1.23 +- 0.53
1.23 +- 0.97
